<a href="https://colab.research.google.com/github/sulaimanbehzad/Classifying-Images/blob/main/Image_Classifiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classifiers
The purpose of this project is to train two classifiers:
1. Captions classifier
2. Image classifier

In [6]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

import imageio
import cv2

## Part 2: Image Classifier

Reading the images and format them into a dataframe

In [9]:
# function for reading images into a pandas dataframe
def read_data_into_df(filespath):
  levels = 0
  default_size = 256
  for roots, dirnames, filenames in os.walk(filespath):
    if levels == 0:
      df = pd.DataFrame(columns=dirnames)
    im_list = []
    for fn in filenames:
      im_full_path = os.path.join(roots, fn)
      # print(txtfile_full_path)
      # temp=pd.read_csv(txtfile_full_path,sep="\\n", header=None, error_bad_lines=False)
      # print('shape of temp: ', temp.shape)
      # txt_list.append(temp.values)
      # -------------------- another approach to read txt files
      im = cv2.imread(im_full_path)[...,::-1] 
      im_resized = cv2.resize(im, (default_size, default_size))
      im_list.append(im_resized)
    root = os.path.split(roots) 
    root = root[1]
    # print(root)
    if levels != 0:
      df[root] = im_list
    levels+=1
  return df


In [ ]:
path_train_sentences = r'/content/drive/MyDrive/dataset/train/images'   
path_test_sentences =  r'/content/drive/MyDrive/dataset/test/images'   
train = read_data_into_df(path_train_sentences)
test = read_data_into_df(path_test_sentences)

### Dataframe inspection, evaluation and preprocessing


In [ ]:
train.head()

In [ ]:
test.head()

The dataframes don't have any null values so we are good to go on that aspect  
What remains is to add tags to the captions of each type

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
for i in train['chair']:
  print(i)

In [ ]:
def make_tabular_df(df):
  tags = df.columns
  df_transpose = df.transpose(copy=False)
  output = pd.DataFrame(columns=['captions',  'tags'])
  for index, col in df_transpose.iteritems():
    new = []
    for item in col:
      new.append(item)
    df=pd.DataFrame({"captions": new, "tags": tags})
    output=output.append(df)
  return output


In [ ]:
X_train = make_tabular_df(train)
X_train.head()

In [ ]:
X_test = make_tabular_df(test)
X_test.head()

We obtained the dataframes ready to go to next phase


In [ ]:
plt.figure(figsize=(24,20))
X_train.tags.value_counts().plot(kind='bar', color=['Blue', 'Red', 'Brown', 'Gray']);

We can confirm visually that our dataset is balanced

## Bag-Of-Words (BOW) with Keras

In [ ]:
print(f"train size: {len(X_train)}")
print(f"test size: {len(X_test)}")

In [ ]:
max_words = 1500
tokenizer = text.Tokenizer(num_words=max_words, char_level=False)

In [ ]:
tokenizer.fit_on_texts(X_train['captions']) 
x_train = tokenizer.texts_to_matrix(X_train['captions'])
x_test = tokenizer.texts_to_matrix(X_test['captions'])
x_train

In [ ]:
x_test

In [ ]:
print(f'shape of new x_train is: {x_train.shape}')
print(f'shape of new x_test is: {x_test.shape}')

in both encoder and tokenizer we only fit on train for consistency

In [ ]:
le = LabelEncoder()
le.fit(X_train['tags'])
y_train = le.transform(X_train['tags'])
y_test = le.transform(X_test['tags'])

In [ ]:
print(f'shape of new y_train is: {y_train.shape}')
print(f'shape of new y_test is: {y_test.shape}')

Convert the enoded y_train and y_test to one-hot representation

In [ ]:
num_tags = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_tags)
y_test = utils.to_categorical(y_test, num_tags)
y_train

In [ ]:
y_test

In [ ]:
print(f'shape of new x_train is: {x_train.shape}')
print(f'shape of new x_test is: {x_test.shape}')
print(f'shape of new y_train is: {y_train.shape}')
print(f'shape of new y_test is: {y_test.shape}')

Let's set our hyperparameters  
we will adjust these later to, if possible, reach higher accuracy

In [ ]:
batch_size = 16
epochs = 3

Buliding the model

In [ ]:
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_tags))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

we have the model, train and test data prepared  
so let's train the model

In [ ]:
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
                    verbose=1, validation_split=0.1)

In [ ]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print(f'Test score:{score[0]}')
print(f'Test accuracy:{score[1]}')

In [ ]:
txt_lbls = le.classes_ 

for i in range(10):
    prediction = model.predict(np.array([x_test[i]]))
    predicted_label = txt_lbls[np.argmax(prediction)]
    print(X_test['captions'].iloc[i][:50], "...")
    print('Actual label:' + X_test['tags'].iloc[i])
    print("Predicted label: " + predicted_label + "\n")

In [ ]:
# This utility function is from the sklearn docs: http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
import itertools
def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """

    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=30)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, fontsize=22)
    plt.yticks(tick_marks, classes, fontsize=22)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label', fontsize=25)
    plt.xlabel('Predicted label', fontsize=25)

In [ ]:
y_sm = model.predict(x_test)

y_test_1d = []
y_pred_1d = []

for i in range(len(y_test)):
    probs = y_test[i]
    index_arr = np.nonzero(probs)
    one_hot_index = index_arr[0].item(0)
    y_test_1d.append(one_hot_index)

for i in range(0, len(y_sm)):
    probs = y_sm[i]
    predicted_index = np.argmax(probs)
    y_pred_1d.append(predicted_index)

In [ ]:
cnf_matrix = confusion_matrix(y_test_1d, y_pred_1d)
plt.figure(figsize=(24,20))
plot_confusion_matrix(cnf_matrix, classes=txt_lbls, title="Confusion matrix")
plt.show()